In [1]:
import json
import pandas as pd
import numpy as np
import autosklearn

# Jsons to get Team numbers for Opponents and URLs to get team schedule

In [2]:
with open('team_schedule.json') as f:
    data1 = json.load(f)
team_key = list(data1.keys())
team_url = list(data1.values())
f.close()

In [3]:
with open('team_numbers.json') as f:
    data = json.load(f)
team_name = list(data.keys())
team_number = list(data.values())
f.close()

# Place Teams in Conferences Finals into dataframes

In [4]:
Lakers = pd.read_html(data1['LAL'],skiprows=(21,42,63))[0]
Denver = pd.read_html(data1['DEN'],skiprows=(21,42,63))[0]
Heat = pd.read_html(data1['MIA'],skiprows=(21,42,63))[0]
Boston = pd.read_html(data1['BOS'],skiprows=(21,42,63))[0]

In [5]:
Lakers = Lakers[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]
Denver = Denver[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]
Heat = Heat[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]
Boston = Boston[['Tm','Opp','Unnamed: 5','Unnamed: 7','Opponent']]

In [6]:
list_teams = [Lakers,Denver,Heat,Boston]

<b>Change Name to Home/Away. Change Home games to = 1 and Away values = 0 </b>

In [7]:
Lakers.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)
Denver.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)
Heat.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)
Boston.rename({'Unnamed: 5': 'Home/Away'}, axis=1,inplace=True)

In [8]:
#Can only run once or Home/Away data will say all 1s
for i in list_teams:
    list2 = []
    for index,row in i.iterrows():
        if row['Home/Away'] == '@':
            list2.append(0)
        else:
            list2.append(1)
    i['Home/Away'] = list2

<b>Create Column for Opponent team numercial value which is determined in team_numbers.json </b>

In [9]:
Lakers['Team_num'] = 0
Denver['Team_num'] = 0
Heat['Team_num'] = 0
Boston['Team_num'] = 0

In [10]:
for i in list_teams:
    list_ = []
    for index,row in i.iterrows():
        if row.Opponent in data:
            list_.append(data[row.Opponent])
    i['Team_num'] = list_

<b>Create X and Y axis for each of the teams to run in the model</b>

In [11]:
Lakers['Home'] = 16
Denver['Home'] = 18
Heat['Home'] = 3
Boston['Home'] = 5

In [12]:
Lakers_x = Lakers[["Home","Team_num","Home/Away"]].to_numpy()
Lakers_y = Lakers[["Tm","Opp"]].to_numpy()
Lakers_series = Lakers.to_numpy()

In [13]:
Denver_x = Denver[["Home","Team_num","Home/Away"]].to_numpy()
Denver_y = Denver[["Tm","Opp"]].to_numpy()
Denver_series = Denver.to_numpy()

In [14]:
Heat_x = Heat[["Home","Team_num","Home/Away"]].to_numpy()
Heat_y = Heat[["Tm","Opp"]].to_numpy()
Heat_series = Heat.to_numpy()

In [15]:
Boston_x = Boston[["Home","Team_num","Home/Away"]].to_numpy()
Boston_y = Boston[["Tm","Opp"]].to_numpy()
Boston_series = Boston.to_numpy()

# Linear Regression Model

In [16]:
import numpy as np
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True, normalize=True)

# Train the model
model = model.fit(Lakers_x,Lakers_y)

_sq = model.score(Lakers_x, Lakers_y)

print(_sq)

0.07021453321344545


In [17]:
nuggets = model.predict(np.array([[16,18,1],[16,18,0]]))
print(nuggets)

[[114.04014844 106.39708354]
 [113.37096202 108.9271338 ]]


# Try using KNN Algorithm

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(Lakers_x, Lakers_y)
nuggets = knn.predict(np.array([[16,18,1],[16,18,0]]))
print(nuggets)

[[ 97 121]
 [124  96]]
